In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qutip as qp
from tqdm import tqdm
from matplotlib.animation import FuncAnimation
from copy import deepcopy
import copy
from itertools import combinations


from mindquantum import Circuit, UN, H, Z, X, Measure
from mindquantum.simulator import Simulator
plt.rcParams["figure.dpi"]=120

In [3]:
n_qubit = 4

# possible
z_combinations = []
control_z_combinations = []
control_2_z_combinatons = []
control_3_z_combinations = []

## 排列组合
qubits_list = np.arange(0, n_qubit, 1)
z_all = [1]*n_qubit
control_z_all = list(combinations(qubits_list,2))
control_2_z_all = list(combinations(qubits_list,3))
control_3_z_all = list(combinations(qubits_list,4))

for i in range(2**len(z_all)):
    z_combination = list('{:04b}'.format(i))
    z_combinations.append(list(map(int, z_combination)))

for i in range(2**len(control_z_all)):
    control_z_combination = list('{:06b}'.format(i))
    control_z_combination = list(map(int, control_z_combination))
    for index, possible in enumerate(control_z_combination):
        if possible != 0: control_z_combination[index] = list(control_z_all[index])
    control_z_combinations.append(control_z_combination)  
    
for i in range(2**len(control_2_z_all)):
    control_2_z_combinaton = list('{:04b}'.format(i))
    control_2_z_combinaton = list(map(int, control_2_z_combinaton))
    for index, possible in enumerate(control_2_z_combinaton):
        if possible != 0: control_2_z_combinaton[index] = list(control_2_z_all[index])
    control_2_z_combinatons.append(control_2_z_combinaton)
    
for i in range(2**len(control_3_z_all)):
    control_3_z_combination = list('{:01b}'.format(i))
    control_3_z_combination = list(map(int, control_3_z_combination))
    for index, possible in enumerate(control_3_z_combination):
        if possible != 0: control_3_z_combination[index] = list(control_3_z_all[index])
    control_3_z_combinations.append(control_3_z_combination) 

## Generate Hypergaph state

In [4]:
def Quantum_Hypergraph_cicuits(z_position, control_z_positon, control_2_z_positon, control_3_z_positon, flag):
    
    n_qubits = 4                                # 设定量子比特数为3
    sim = Simulator('projectq', n_qubits)        # 使用projectq模拟器，命名为sim
    circuit = Circuit()                          # 初始化量子线路，命名为circuit

    if flag == True:
         circuit += UN(H, n_qubits) 
        
    for index, position in enumerate(z_position):
        if position == 1:
            circuit += Z.on([index])

    for index, position in enumerate(control_z_positon):
        if isinstance(position,list):
            target = int(position[1])
            control = int(position[0])
            circuit += Z.on([target], [control])
            
    for  index, position in enumerate(control_2_z_positon):
        if isinstance(position,list):
            target = int(position[-1])
            control = [int(i) for i in position]
            del control[-1]
            circuit += Z.on([target], control)

    for index, position in enumerate(control_3_z_positon):
        if isinstance(position,list):
           circuit += Z.on([3], [0, 1, 2])
    return circuit


def Quantum_Hypergraph_cicuits_symmetry(z_position, control_z_positon, control_2_z_positon, control_3_z_positon, flag):

    # To get the global phase -1
    one_control = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
    two_control = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)] 

    n_qubits = 4                                          # 设定量子比特数为3
    sim = Simulator('projectq', n_qubits)        # 使用projectq模拟器，命名为sim
    circuit = Circuit()                                   # 初始化量子线路，命名为circuit

    if flag == True:
        circuit += UN(H, n_qubits) 
    circuit += UN(Z, n_qubits) 

    for index, value in enumerate(one_control):
            circuit += Z.on([value[1]], [value[0]])

    for index, value in enumerate(two_control):
        circuit += Z.on([value[-1]], [value[0], value[1]])

    circuit += Z.on([3], [0,1,2])
        
    circuit += UN(X, n_qubits) 
    circuit += Z.on([3], [0,1,2])
    
    for index, position in enumerate(z_position):
        if position == 1:
            circuit += Z.on([index])

    for index, position in enumerate(control_z_positon):
        if isinstance(position,list):
            target = int(position[1])
            control = int(position[0])
            circuit += Z.on([target], [control])
            
    for  index, position in enumerate(control_2_z_positon):
        if isinstance(position,list):
            target = int(position[-1])
            control = [int(i) for i in position]
            del control[-1]
            circuit += Z.on([target], control)

    for index, position in enumerate(control_3_z_positon):
        if isinstance(position,list):
           circuit += Z.on([3], [0, 1, 2])
            
    return circuit


In [5]:
 Quantum_Hypergraph_cicuits(z_combinations[2], control_z_combinations[2],control_2_z_combinatons[3],control_3_z_combinations[0], True)

q0: ──H──────────────●───────
                     │
q1: ──H─────────●────┼────●──
                │    │    │
q2: ──H────Z────┼────●────●──
                │    │    │
q3: ──H─────────Z────Z────Z──

In [6]:
 Quantum_Hypergraph_cicuits_symmetry(z_combinations[0], control_z_combinations[0],control_2_z_combinatons[0],control_3_z_combinations[0], True)


q0: ──H────Z────●────●────●───────────────────●────●────●─────────●────X────●──
                │    │    │                   │    │    │         │         │
q1: ──H────Z────Z────┼────┼────●────●─────────●────●────┼────●────●────X────●──
                     │    │    │    │         │    │    │    │    │         │
q2: ──H────Z─────────Z────┼────Z────┼────●────Z────┼────●────●────●────X────●──
                          │         │    │         │    │    │    │         │
q3: ──H────Z──────────────Z─────────Z────Z─────────Z────Z────Z────Z────X────Z──

In [7]:
n_qubits = 4
sim = Simulator('projectq', n_qubits)        # 使用projectq模拟器，命名为sim
sim.reset() 
circ =  Quantum_Hypergraph_cicuits_symmetry(z_combinations[0], control_z_combinations[0],control_2_z_combinatons[0],control_3_z_combinations[0], True)
sim.apply_circuit(circ)                       # 通过模拟器sim运行搭建好的量子线路circuit
result = sim.get_qs(True)                      # 打印模拟器sim中运行量子线路circuit后的末态


In [8]:
data_circuits_input = []
data_circuits_weight = []
for c_3_z_gate in control_3_z_combinations:
    for c_2_z_gate in control_2_z_combinatons:
        for c_z_gate in control_z_combinations:
            for z_gate in z_combinations:
                data_circuits_input.append(Quantum_Hypergraph_cicuits(z_gate, c_z_gate, c_2_z_gate, c_3_z_gate,True))
                data_circuits_input.append(Quantum_Hypergraph_cicuits_symmetry(z_gate, c_z_gate, c_2_z_gate, c_3_z_gate,True))
                data_circuits_weight.append(Quantum_Hypergraph_cicuits(z_gate, c_z_gate, c_2_z_gate, c_3_z_gate, False))
                data_circuits_weight.append(Quantum_Hypergraph_cicuits_symmetry(z_gate, c_z_gate, c_2_z_gate, c_3_z_gate, False))

                
circuits_state = []
for circ in data_circuits_input:

    n_qubits = 4
    sim = Simulator('projectq', n_qubits)        # 使用projectq模拟器，命名为sim
    sim.reset() 
    sim.apply_circuit(circ)                       # 通过模拟器sim运行搭建好的量子线路circuit
    result = sim.get_qs(True)  
    state = []
    for i , value in enumerate(result.splitlines()):
        if value[0] == '-':  state.append(0);
        else:  state.append(1);
    state = tuple(state)
    circuits_state.append(state)

## Circuits_state----Quantum Circuits

In [9]:
input_dataset = dict(zip(circuits_state, data_circuits_input))  ### 全部的
weight_dataset = dict(zip(circuits_state, data_circuits_weight)) 
len(input_dataset)
set_lst=set(input_dataset)
len(set_lst)

65536

## Generate data Lable

In [10]:
w_target = np.array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1])
w_target_symmetry = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
number = np.array([6,9,10,11,14])
possible = list(combinations(number,2))

    
positive_circuit = []
for index,value in enumerate(w_target):
    data_copy = copy.deepcopy(w_target)
    data_symmergy_copy = copy.deepcopy(w_target_symmetry)
    if index == 6 or index == 9 or index == 10 or index == 11 or index == 14: # 少一个
        data_copy[index] = 1
        data_symmergy_copy[index] = 0
        positive_circuit.append(tuple(data_copy))
        positive_circuit.append(tuple(data_symmergy_copy))
    if index == 5 or index == 7 or index == 13 or index == 15: # 多一个
        data_copy[index] = 0
        positive_circuit.append(tuple(data_copy))
        
for i in possible:
    data_copy = copy.deepcopy(w_target)
    data_symmergy_copy = copy.deepcopy(w_target_symmetry)
    data_copy[list(i)[0]] = 1
    data_copy[list(i)[1]] = 1
    data_symmergy_copy[list(i)[0]] = 0
    data_symmergy_copy[list(i)[1]] = 0
    positive_circuit.append(tuple(data_copy))
    positive_circuit.append(tuple(data_symmergy_copy))

# element = [[5,6],[5,9],[7,6],[7,11],[13,9],[13,14],[15,11],[15,14]]
# for index,value in enumerate(w_target):  # 少一个多一个
#     data_copy = copy.deepcopy(w_target)
#     if index == 5 or index == 7 or index == 13 or index == 15 :
#         data_copy[index] = 0
#         for j in element:
#             if j[0] == 5:
#                 data_copy[j[1]] = 1
#                 positive_circuit.append(tuple(data_copy))

positive_circuit.append(tuple(w_target))

In [11]:
global traing_label
   
input_dataset2 = copy.copy(input_dataset)
#positive_index.sort(reverse=True)
training_positive_circuits = {}
traing_label = {}
traing_negtive_circuits = {}
traing_negtive_label = {}

for i in positive_circuit:
    training_positive_circuits[i] = input_dataset2[i]
    traing_label[i] = 1
    
for i in positive_circuit:
    del input_dataset2[i]
    
traing_datas_negtive_key = random.sample(input_dataset2.keys(),1000)
for key in traing_datas_negtive_key:
    traing_negtive_circuits[key] = input_dataset2[key]
    traing_negtive_label[key] = 0

training_positive_circuits.update(traing_negtive_circuits)
traing_label.update(traing_negtive_label)

In [12]:
def random_dic(dicts):
    dict_key_ls = list(dicts.keys())
    random.shuffle(dict_key_ls)
    new_dic = {}
    for key in dict_key_ls:
        new_dic[key] = dicts.get(key)
    return new_dic


training_data = random_dic(training_positive_circuits)

In [13]:
#len(training_data)
len(traing_label)

1035

In [20]:
def active_function(input_circuits, weight_circuits):    

    n_qubits = 5                                      # 设定量子比特数为3
    sim = Simulator('projectq', n_qubits)        # 使用projectq模拟器，命名为sim
    circuit = Circuit()                                   # 初始化量子线路，命名为circuit

    circuit += input_circuits
    circuit += weight_circuits
    circuit += UN(H, n_qubits-1) 
    circuit += UN(X, n_qubits-1) 
    circuit += X.on([4], [0, 1, 2, 3])
    circuit += Measure().on(4)

    res = sim.sampling(circuit, shots=5000)
    counts = res.data
    key_number = []
    for i in counts.keys():
        key_number.append(i)
    if len(key_number) == 2: 
        one_weight = counts["1"]/(counts["0"]+counts["1"])
    elif len(key_number) == 1: 
        for key, value in counts.items():
            if key == '1':
                one_weight = value/5000
            else:
                one_weight = 0
                
    if one_weight > 0.5:  ### Threshoud value
        pred = 1 
    else: pred = 0
    return pred


def update(in_vector, w_vector, pred):
    global correct
    global label
    
    w_vector = w_vector
    
    if pred == 1 and traing_label[in_vector]==1:
        correct += 1
        return w_vector, correct
        
    elif pred == 0 and traing_label[in_vector]==0:
        correct += 1
        return w_vector, correct

    elif pred == 1 and traing_label[in_vector]==0:     ## move far away
        
        while 1:
            wrong = 0
            for i in range(16):
                if in_vector[i] != w_vector[i]:
                    wrong += 1
            if wrong == 16:
                if w_vector[random.randrange(0, 15,1)] == 0:
                    w_vector = list(w_vector)
                    w_vector[random.randrange(0, 15,1)] = 1
                    w_vector = tuple(w_vector)
                else:
                    w_vector = list(w_vector)
                    w_vector[random.randrange(0, 15,1)] = 0
                    w_vector = tuple(w_vector)
            w_vector = list(w_vector)
            w_vector = list(w_vector)
            
            for i in range(16):
                if w_vector[i] == in_vector[i]:  w_vector[i] = random.randrange(0, 2,1);

            w_vector = tuple(w_vector)
            input_circuits = input_dataset[in_vector]
            weight_circuits = weight_dataset[w_vector]
            
            pred = active_function(input_circuits, weight_circuits)  
            if pred == 0:
                correct += 1
                break
        return w_vector, correct
    
    elif pred == 0 and traing_label[in_vector]==1:    ## move far away
        while 1:
            if in_vector == w_vector:
                if w_vector[random.randrange(0, 15,1)] == 0:
                    w_vector = list(w_vector)
                    w_vector[random.randrange(0, 15,1)] = 1
                    w_vector = tuple(w_vector)
                else:
                    w_vector = list(w_vector)
                    w_vector[random.randrange(0, 15,1)] = 0
                    w_vector = tuple(w_vector)
            w_vector = list(w_vector)
            for i in range(16):
                if w_vector[i] != in_vector[i]: w_vector[i] = random.randrange(0, 2,1); 

            w_vector = tuple(w_vector)
            input_circuits = input_dataset[in_vector]
            weight_circuits = weight_dataset[w_vector]
                
            pred = active_function(input_circuits, weight_circuits)  
            if pred == 1:
                correct += 1
                break 
        return w_vector, correct

In [21]:
w_vector = (1,0,0,1,1,1,0,1,1,0,1,0,1,0,0,1)
correct = 0
weight = []

#for i in range(100):
for in_vector in tqdm(training_data):
    input_circuits = input_dataset[in_vector]
    weight_circuits = input_dataset[w_vector]
    pred = active_function(input_circuits, weight_circuits)
    w_vector, correct = update(in_vector, w_vector, pred)
    weight.append(w_vector)
    
weights = []
for i in weight:
    weights.append(np.array(list(i)).reshape((4,4)))

17%|█▋        | 181/1035 [27:27<2:09:33,  9.10s/it]


KeyboardInterrupt: 